In [4]:
import pandas as pd
import re
import argparse
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

pd.set_option("display.max_columns", None)
input_folder = '../data/train'
# Reading input folder
files = os.listdir(input_folder)

#df_std = df_std.append(tmp.std(), ignore_index=True)

file_name = 'df.pkl'

input_folder = '../data/train'
# Reading input folder
filesnames = os.listdir(input_folder)
regex = re.compile(r'\d+')
ids = [int(x) for x in regex.findall(str(files))]
dfs = list()

for patient_id, filename in enumerate(filesnames):    
    pdf = pd.read_csv(input_folder + "/" + filename, sep='|')   
    sepsislabel_true_list = pdf[pdf['SepsisLabel'] == 1].index
    if not sepsislabel_true_list.empty:
        pdf = pdf[:min(pdf[pdf['SepsisLabel'] == 1].index)+1]
        pdf['SepsisLabel'] = 1
    pdf['patient_id'] = ids[patient_id]
    pdf.interpolate(limit_direction='both', axis=0, inplace=True) # show first hist without it
    dfs.append(pdf)
df = pd.concat(dfs, axis=0, ignore_index=True)
df.to_pickle(file_name) # where to save it usually as a .plk

In [5]:
file_name = 'df.pkl'

# Save dataframe to pickled pandas object
#df.to_pickle(file_name) # where to save it usually as a .plk

# Load dataframe from pickled pandas object
df = pd.read_pickle(file_name)


In [7]:

# vital signal features
vitals = ['HR','O2Sat','Temp','SBP','MAP','DBP','Resp','EtCO2']

# lab test features
labs = ['BaseExcess','HCO3','FiO2','pH','PaCO2','SaO2','AST','BUN','Alkalinephos','Calcium','Chloride','Creatinine','Bilirubin_direct','Glucose','Lactate',
      'Magnesium','Phosphate','Potassium','Bilirubin_total','TroponinI','Hct','Hgb','PTT','WBC','Fibrinogen','Platelets']

# demographic features
demogs = ['Age','Gender','Unit1','Unit2','HospAdmTime','ICULOS']

# labels
labels = ['SepsisLabel']

all = vitals+labs+demogs+labels


Try removing all features where misigness is over 90% - 

In [6]:
missingover90 = ['EtCO2', 'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST','BUN', 'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine','Bilirubin_direct', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets']

In [16]:
df.drop(missingover90,axis=1, inplace=True)

In [28]:
set(max_agg).difference(missingover90)

{'Age',
 'Gender',
 'Glucose',
 'HospAdmTime',
 'ICULOS',
 'Resp',
 'SepsisLabel',
 'Temp',
 'Unit1',
 'Unit2',
 'patient_id'}

Option 1 - Full features use

In [8]:
agg = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct','Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets','Age']

aggdict = {}
for col in df.columns:
    if col in agg:
        aggdict[col]=['mean','std', 'min', 'max','skew',pd.DataFrame.kurt]
    else:
        aggdict[col]=['max']

Option 2

In [44]:
agg = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct','Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets','Age']
max_agg = ['Lactate','WBC','Glucose','Temp','Resp','Creatinine','BUN'] + demogs + ['patient_id','SepsisLabel']
min_agg = ['SBP']

aggdict = {}
for col in df.columns:
    if col in min_agg:
        aggdict[col]=['min']
    elif col in max_agg:
        aggdict[col]=['max']
    else:
        aggdict[col]=['mean','std', 'min', 'max','skew',pd.DataFrame.kurt]

Option 3

In [58]:
agg = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct','Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets','Age']
max_agg = ['Lactate','WBC','Glucose','Temp','Resp','Creatinine','BUN'] + demogs + ['patient_id','SepsisLabel']
min_agg = ['SBP']

aggdict = {}
for col in df.columns:
    if col in min_agg:
        aggdict[col]=['min']
    elif col in max_agg:
        aggdict[col]=['max']
    else:
        aggdict[col]=['mean','std','skew',pd.DataFrame.kurt]

Option 4 - The best Features Selection

In [8]:
agg = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct','Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets','Age']
max_agg = ['Lactate','WBC','Glucose','Temp','Resp','Creatinine','BUN'] + demogs + ['patient_id','SepsisLabel']
min_agg = ['SBP']

aggdict = {}
for col in df.columns:
    if col in min_agg:
        aggdict[col]=['min']
    elif col in max_agg:
        aggdict[col]=['max']
    else:
        aggdict[col]=['mean','std']

In [9]:
grouped_multiple = df.groupby('patient_id').agg(aggdict)

#grouped_multiple = df.groupby('patient_id')[columns].agg(['mean','std', 'min', 'max'])

grouped_multiple.columns = list(map('_'.join, grouped_multiple.columns.values))
grouped_multiple

,HR_mean,HR_std,O2Sat_mean,O2Sat_std,Temp_max,SBP_min,MAP_mean,MAP_std,DBP_mean,DBP_std,Resp_max,EtCO2_mean,EtCO2_std,BaseExcess_mean,BaseExcess_std,HCO3_mean,HCO3_std,FiO2_mean,FiO2_std,pH_mean,pH_std,PaCO2_mean,PaCO2_std,SaO2_mean,SaO2_std,AST_mean,AST_std,BUN_max,Alkalinephos_mean,Alkalinephos_std,Calcium_mean,Calcium_std,Chloride_mean,Chloride_std,Creatinine_max,Bilirubin_direct_mean,Bilirubin_direct_std,Glucose_max,Lactate_max,Magnesium_mean,Magnesium_std,Phosphate_mean,Phosphate_std,Potassium_mean,Potassium_std,Bilirubin_total_mean,Bilirubin_total_std,TroponinI_mean,TroponinI_std,Hct_mean,Hct_std,Hgb_mean,Hgb_std,PTT_mean,PTT_std,WBC_max,Fibrinogen_mean,Fibrinogen_std,Platelets_mean,Platelets_std,Age_max,Gender_max,Unit1_max,Unit2_max,HospAdmTime_max,ICULOS_max,SepsisLabel_max,patient_id_max
patient_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,60.956522,7.957148,97.086957,2.130152,36.44,114.0,66.630435,14.386087,41.565217,6.801244,27.0,NaN,NaN,NaN,NaN,22.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,7.900000,0.000000,113.000000,0.000000,2.5,NaN,NaN,78.0,NaN,2.500000,0.000000,4.400000,0.000000,5.100000,0.000000,NaN,NaN,NaN,NaN,27.800000,0.000000,9.700000,0.000000,NaN,NaN,11.0,NaN,NaN,158.000000,0.000000,75.91,0,0.0,1.0,-98.60,23,0,0
1,87.640000,6.885250,100.000000,0.000000,38.17,103.0,87.660000,8.121114,63.740000,5.285830,22.0,NaN,NaN,0.360000,0.422222,23.160000,0.336473,0.412000,0.043970,7.430800,0.008164,35.640000,1.321534,NaN,NaN,NaN,NaN,17.0,NaN,NaN,8.388000,0.161186,109.000000,0.000000,1.0,NaN,NaN,150.0,2.2,2.344000,0.080593,3.984000,0.120890,3.968000,0.241779,NaN,NaN,NaN,NaN,33.784000,1.527520,12.292000,0.220088,25.292000,0.564151,12.5,NaN,NaN,196.200000,16.118608,65.79,1,NaN,NaN,-0.02,26,1,1
2,75.000000,7.830344,95.916667,2.205513,37.67,111.0,93.264167,8.004568,NaN,NaN,23.0,NaN,NaN,NaN,NaN,27.000000,0.000000,0.500000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,103.0,0.0,13.0,108.0,0.0,NaN,NaN,106.000000,0.000000,1.2,NaN,NaN,145.0,NaN,1.600000,0.000000,NaN,NaN,3.400000,0.000000,1.7,0.0,NaN,NaN,31.500000,0.000000,11.700000,0.000000,32.300000,0.000000,3.5,NaN,NaN,65.000000,0.000000,52.88,1,1.0,0.0,-0.01,38,0,2
3,75.988889,10.617572,96.711111,1.547501,37.78,113.0,89.111111,8.133464,70.644444,7.461928,21.0,NaN,NaN,NaN,NaN,23.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,8.775556,0.145457,107.933333,1.090926,0.6,NaN,NaN,244.0,NaN,2.000000,0.000000,2.593333,0.109093,3.900000,0.000000,NaN,NaN,NaN,NaN,41.591111,0.945469,14.188889,0.363642,22.280000,0.327278,11.6,NaN,NaN,211.888889,36.364185,49.52,1,NaN,NaN,-0.02,48,0,3
4,98.114583,5.540355,95.250000,1.793709,37.00,104.0,82.604167,4.104555,63.958333,4.885464,23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN,7.800000,0.000000,NaN,NaN,1.5,NaN,NaN,233.0,NaN,2.041667,0.046780,NaN,NaN,3.991667,0.233902,NaN,NaN,NaN,NaN,35.300000,0.000000,11.300000,0.000000,NaN,NaN,10.8,NaN,NaN,170.000000,0.000000,73.00,1,1.0,0.0,-214.64,24,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,104.103448,11.241171,97.827586,1.769160,38.00,98.0,85.327586,9.810959,66.948276,5.477788,24.0,NaN,NaN,2.879310,0.295960,NaN,NaN,0.456897,0.164601,7.393103,0.013683,46.517241,1.183840,98.12069,0.295960,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,0.7,NaN,NaN,169.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.672414,1.821818,11.151724,0.553566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.71,1,0.0,1.0,-4.12,29,0,19995
19996,63.767442,6.089950,99.651163,0.650411,37.22,114.0,102.767442,31.519334,75.697674,20.106307,23.0,NaN,NaN,6.325581,1.783711,27.720930,0.361652,0.511628,0.108060,7.479767,0.028448,40.941860,2.106808,NaN,NaN,NaN,NaN,58.0,NaN,NaN,8.104651,0.253156,95.720930,0.361652,8.2,NaN,NaN,164.0,1.0,2.07209

Consider Feature selection

In [10]:
train = grouped_multiple.drop(['SepsisLabel_max','patient_id_max'], axis=1)
train=train.fillna(train.mean())
y = grouped_multiple['SepsisLabel_max']

In [72]:
sfs = SequentialFeatureSelector(LogisticRegression(), scoring='f1', n_jobs=-1)
sfs = sfs.fit(train, y)
chosen = train.columns[sfs.support_]
X_t = pd.DataFrame(sfs.transform(train), columns=chosen, index=train.index)

In [73]:
chosen

Index(['O2Sat_mean', 'O2Sat_std', 'DBP_mean', 'DBP_std', 'Resp_max',
       'BaseExcess_mean', 'BaseExcess_std', 'FiO2_std', 'pH_mean', 'pH_std',
       'PaCO2_std', 'SaO2_mean', 'AST_mean', 'Alkalinephos_mean',
       'Calcium_mean', 'Chloride_mean', 'Bilirubin_direct_mean',
       'Bilirubin_direct_std', 'Glucose_max', 'Lactate_max', 'Phosphate_mean',
       'Phosphate_std', 'Potassium_mean', 'Bilirubin_total_std',
       'TroponinI_mean', 'Hct_std', 'Hgb_std', 'Fibrinogen_mean',
       'Fibrinogen_std', 'Age_max', 'Unit2_max', 'HospAdmTime_max',
       'ICULOS_max'],
      dtype='object')

In [85]:
features = list(chosen) +list(['SepsisLabel_max']) + list(['patient_id_max'])

In [86]:
grouped_multiple = grouped_multiple[features]

# Prediction

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import MissingIndicator
from feature_engine import imputation

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
grouped_multiple.drop(['SepsisLabel_max','patient_id_max'], axis=1), grouped_multiple['SepsisLabel_max'], test_size=0.2,
random_state=0)

In [13]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.mixture import GaussianMixture
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
from sklearn.impute import KNNImputer


In [14]:
Imputer = SimpleImputer(strategy="mean", add_indicator=True)
impute_scale_pipeline = make_pipeline(Imputer, StandardScaler())

In [50]:
rf_pipe_scale = make_pipeline(impute_scale_pipeline, RandomForestClassifier())
svc_pipe_scale = make_pipeline(impute_scale_pipeline, SVC())
gb_pipe_scale = make_pipeline(impute_scale_pipeline, GradientBoostingClassifier())
dt_pipe_scale = make_pipeline(impute_scale_pipeline, DecisionTreeClassifier() )
mlp_pipe_scale = make_pipeline(impute_scale_pipeline, MLPClassifier() )

In [51]:
scores_rf_pipe_scale = cross_validate(rf_pipe_scale, X_train, 
                                             y_train, 
                                             cv=StratifiedKFold(3, shuffle=True)
                                             ,scoring=["f1", 
                                                       "roc_auc", "precision", 
                                                       "recall"])



scores_gb_pipe_scale = cross_validate(gb_pipe_scale, X_train, 
                                             y_train, 
                                             cv=StratifiedKFold(3, shuffle=True)
                                             ,scoring=["f1", 
                                                       "roc_auc", "precision", 
                                                       "recall"])



scores_dt_pipe_scale = cross_validate(dt_pipe_scale, X_train, 
                                             y_train, 
                                             cv=StratifiedKFold(3, shuffle=True)
                                             ,scoring=["f1", 
                                                       "roc_auc", "precision", 
                                                       "recall"])


scores_mlp_pipe_scale = cross_validate(mlp_pipe_scale, X_train, 
                                             y_train, 
                                             cv=StratifiedKFold(3, shuffle=True)
                                             ,scoring=["f1", 
                                                       "roc_auc", "precision", 
                                                       "recall"])



In [17]:
from xgboost.sklearn import XGBClassifier

params = {'xgbclassifier__colsample_bytree': 1.0,
          'xgbclassifier__gamma': 1,
          'xgbclassifier__max_depth': 3,
          'xgbclassifier__min_child_weight': 1,
          'xgbclassifier__subsample': 1.0}

XGB_pipe_scale = make_pipeline(impute_scale_pipeline, XGBClassifier(params=params))


scores_xgboost= cross_validate(XGB_pipe_scale, X_train, 
                                             y_train, 
                                             cv=StratifiedKFold(3, shuffle=True)
                                             ,scoring=["f1", 
                                                       "roc_auc", "precision", 
                                                       "recall"])


[19:44:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "params" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:45:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "params" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:45:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "params" } might not be used.

  This could be a false

In [53]:
dt_df = pd.DataFrame(scores_dt_pipe_scale)
dt_df['model'] = 'DecisionTree'

xgb_df = pd.DataFrame(scores_xgboost)
xgb_df['model'] = 'XGBoost'

gb_df = pd.DataFrame(scores_gb_pipe_scale)
gb_df['model'] = 'GradientBoosting'

rf_df = pd.DataFrame(scores_rf_pipe_scale)
rf_df['model'] = 'RandomForest'

mlp_df = pd.DataFrame(scores_mlp_pipe_scale)
mlp_df['model'] = 'MLP'

results = pd.concat([xgb_df,dt_df, gb_df, rf_df, mlp_df])
results

,fit_time,score_time,test_f1,test_roc_auc,test_precision,test_recall,model
0,3.982122,0.166000,0.745342,0.923778,0.898876,0.636605,XGBoost
1,4.817104,0.136550,0.716511,0.911214,0.867925,0.610080,XGBoost
2,4.303745,0.143003,0.693944,0.901395,0.905983,0.562334,XGBoost
0,1.816887,0.063281,0.618807,0.807123,0.591876,0.648305,DecisionTree
1,1.816152,0.048084,0.608081,0.801342,0.581081,0.637712,DecisionTree
2,2.085796,0.065745,0.590631,0.790019,0.567515,0.615711,DecisionTree
0,18.967680,0.068001,0.728395,0.920882,0.872781,0.625000,GradientBoosting
1,18.696950,0.083641,0.710327,0.931927,0.875776,0.597458,GradientBoosting
2,20.002297,0.084206,0.711563,0.922884,0.886076,0.594480,GradientBoosting
0,6.470214,0.430870,0.673884,0.909771,0.932584,0.527542,RandomForest
